## 10.6 自注意力和位置编码
在深度学习中，我们经常使用卷积神经网络(CNN)或循环神经网络(RNN)对序列进行编码。想象一下，有了注意力机制之后，我们将词元序列输入注意力池化中，以便同一组词元同时充当查询、键和值。具体来说，每个查询都会关注所有的键——值对并生成一个注意力输出。由于查询、键和值来自同一组输入，因此被称为自注意力(self-attention)，也被称为内部注意力(intra-attention)。在本节中，我们将使用自注意力进行序列编码，以及如何使用序列的顺序作为补充信息。

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

### 10.6.1 自注意力
给定一个由词元组成的输入序列$x_1, ..., x_n$，其中任意$x_i \in \mathbb R^d (1 ≤ i ≤ n)$。该序列的自注意力输出为一个长度相同的序列$y_1, ..., y_n$，其中：
$$
y_i = f(x_i, (x_1, x_1), ..., (x_n, x_n)) \in \mathbb R^d
\tag{10.6.1}
$$

根据(10.2.4)中定义的注意力池化函数f。下面的代码片段是基于多头注意力对一个张量完成自注意力的计算，张量的形状为(批量大小, 时间步数的数目或词元序列的长度, d)。输出与输入的张量形状相同。

In [3]:
num_hiddens, num_heads = 100, 5
attention = d2l.MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                                    num_hiddens, num_heads, 0.5)
attention.eval()                                    

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [4]:
batch_size, num_queries, valid_lens = 2, 4, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
attention(X, X, X, valid_lens).shape

torch.Size([2, 4, 100])

### 10.6.2 比较卷积神经网络、循环神经网络和自注意力
让我们比较下面几个架构，目标都是将由n个词元组成的序列映射到另一个长度相等的序列，其中的每个输入词元或输出词元都由d维向量表示。具体来说，我们将比较的是卷积神经网络、循环神经网络和自注意力这几个架构的计算复杂性、顺序操作和最大路径长度。请注意，顺序操作会妨碍并行计算，而任意的序列位置组合之间的路径越短，则能更轻松地学习序列中的远距离依赖关系。


